In [1]:
import os
import sys
import glob

import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import pickle

In [ ]:
def readNaluOutput(folder,basename,ts):
    tmp = glob.glob(folder + basename + "*")[0].split(".")

    fullBaseName = tmp[0] + "." + tmp[1] + "."

    numNodes = int(glob.glob(folder + basename + "*")[0].split(basename)[1].split(".")[2])

    fillSize = 4
    if   (numNodes <= 10):    fillSize = 1
    elif (numNodes <= 100):   fillSize = 2
    elif (numNodes <= 1000):  fillSize = 3
    
    for j in range(0,numNodes):
        fn = fullBaseName + str(numNodes) + "." + str(j).zfill(fillSize)
        f = nc.Dataset(fn,"r")
        
        if (j == 0):
            print ("Analysis for time: ", f["time_whole"][ts])
            sys.stdout.write("Reading %s output files..." % numNodes)

        dataFrameDict = {'x':f['coordx'], 'y':f['coordy'], 'z':f['coordz']}
    
        for i in range(0,f.dimensions['num_nod_var'].size):
            varName = f.variables['name_nod_var'][i].data.tostring()
            varName = varName.decode().split('\x00')[0]
            varData = f["vals_nod_var%i" % (i+1)][ts]
            dataFrameDict[varName] = varData[:]
        
        tmpData = pd.DataFrame(dataFrameDict)

        if (j == 0):  rawData = tmpData
        else:         rawData = pd.concat([rawData,tmpData])
        
        sys.stdout.flush()
        sys.stdout.write("\r Finished reading output file %s of %s" % (j+1,numNodes))
    
    return rawData

def readSingleNaluOutput(filename,ts): 
    f = nc.Dataset(filename,"r")
    print ("Analysis for time: ", f["time_whole"][ts])
    dataFrameDict = {'x':f['coordx'], 'y':f['coordy'], 'z':f['coordz']}

    for i in range(0,f.dimensions['num_nod_var'].size):
        varName = f.variables['name_nod_var'][i].data.tostring()
        varName = varName.decode().split('\x00')[0]
        varData = f["vals_nod_var%i" % (i+1)][ts]
        dataFrameDict[varName] = varData[:]

    rawData = pd.DataFrame(dataFrameDict)
    
    return rawData
    
def processNaluChannelTAMS(folder,basename,ts,reTau=180,var=['velocity_x'],returnObject=False):
    naluData = readNaluOutput(folder, basename, ts)
    plotNaluChannelTAMS(naluData,reTau,var,returnObject)
    
def plotNaluChannelTAMS(naluData, reTau=180,var=['velocity_x'],returnObject=False,pltMultiple=False):
    print ("Value for Re_Tau of: ", reTau)
    
    naluData = naluData.round(10)
    naluData["fluctUx"] = naluData["velocity_x"] - naluData["average_velocity_x"]
    naluData["fluctUy"] = naluData["velocity_y"] - naluData["average_velocity_y"]
    naluData["fluctUz"] = naluData["velocity_z"] - naluData["average_velocity_z"]
    naluData["varUx"] = naluData["fluctUx"]*naluData["fluctUx"]
    naluData["varUy"] = naluData["fluctUy"]*naluData["fluctUy"]
    naluData["varUz"] = naluData["fluctUz"]*naluData["fluctUz"]
    
    yCoords = naluData.y.unique()

    ind = 0
    for yc in yCoords:
        sliceY = naluData.loc[(naluData['y'] == yc)]
        planarAvgDict = {'y' : yc*reTau}
        for col in sliceY.columns:
            if ((col == 'x') or (col == 'y') or (col == 'z')):  continue
            planarAvgDict[col] = sliceY[col].sum()/len(sliceY)
            
        tmpFrame = pd.DataFrame(planarAvgDict, index=[ind])
        
        if (ind == 0) :  pltFrame = tmpFrame
        else:            pltFrame = pd.concat([pltFrame,tmpFrame])
            
        ind = ind + 1

    if (pltMultiple):
        return pltFrame
    else:
        plotChannelTAMS([pltFrame], reTau, var, returnObject)
    
def plotChannelTAMS(pltFrames, reTau=180, var=['velocity_x'], returnObject=False):
    for pltFrame in pltFrames:
        pltFrame = pltFrame.loc[(pltFrame['y'] <= reTau)]
        pltFrame = pltFrame.sort_values(by=['y'])
 
    savePath = '/Users/jmelvin/Desktop/WorkFromHome/Re5200Plots'
    save = False
    log = True

    colors = ['b','r','m','c','y','g']
    for varItem in var:
        fig, ax = plt.subplots()
        for index in range(0, len(pltFrames)):
            ax.plot(pltFrames[index].y,pltFrames[index][varItem],colors[index],linewidth=4)
        if ((varItem == "velocity_x") or (varItem == "average_velocity_x")):
            if (reTau == 1000):
                dnsPath = '/Users/blmindell/Desktop/Jeremy/WorkFromHome/Re1000.txt'
            elif (reTau == 5200):
                dnsPath = '/Users/blmindell/Desktop/Jeremy/WorkFromHome/Re5200.txt'
            DNSdata = pd.read_csv(dnsPath, delim_whitespace=True)
            ax.plot(DNSdata["y^+"],DNSdata["U"],'k',linewidth=3)
        elif (varItem == "turbulent_ke"):
            dnsPath = 'tke-Re5200.txt'
            #DNSdata = pd.read_csv(dnsPath, delim_whitespace=True)
            #ax.plot(DNSdata["y^+"],DNSdata["k"],'k',linewidth=3)          
        if (log):
            ax.set_xscale('log')
            ax.set_xlim([1, reTau])

        for label in ax.get_xticklabels():
            label.set_fontsize(16)
        for label in ax.get_yticklabels():
            label.set_fontsize(16)  
        ax.xaxis.label.set_size(16)
        ax.yaxis.label.set_size(16)
        ax.title.set_size(16)
        if (returnObject == False):
            if (save):
                plt.tight_layout()
                plt.savefig(savePath + '/Channel_' + varItem + "_" + str(ts).zfill(4))
                plt.close(fig) 
            else:
                plt.show()
    
    if (returnObject):
        return fig,ax